# <center> Welcome to PyExplainer Quickstart Guide </center>

# PART A - Quick Start

## 1. Prepare data and model

Note. We use the default data and model here for an example

### 1.1 Import required library

In [116]:
from pyexplainer import pyexplainer_pyexplainer

### 1.2 Obtain default dataset and global model (Random Forest)

In [117]:
default_data_and_model = pyexplainer_pyexplainer.get_dflt()
py_explainer = pyexplainer_pyexplainer.PyExplainer(X_train = default_data_and_model['X_train'],
                           y_train = default_data_and_model['y_train'],
                           indep = default_data_and_model['indep'],
                           dep = default_data_and_model['dep'],
                           blackbox_model = default_data_and_model['blackbox_model'])

## 🔧2. Create PyExplainer object 

### 2.1 Prepare data for creating PyExplainer

In [3]:
X_explain = default_data_and_model['X_explain']
y_explain = default_data_and_model['y_explain']

### 2.2 Create rules

In [4]:
created_rules = py_explainer.explain(X_explain=X_explain,
                                     y_explain=y_explain,
                                     search_function='crossoverinterpolation')

## 3. Create interactive visualization

You can change feature values at the slider bar to observe change of risk score.

In [5]:
py_explainer.visualise(created_rules)

Output(layout=Layout(border='3px solid black'))

FloatSlider(value=0.74, continuous_update=False, description='#1 The value of ent is more than 0.74', layout=L…

FloatSlider(value=3.0, continuous_update=False, description='#2 The value of app is less than 3.0', layout=Lay…

FloatSlider(value=3.0, continuous_update=False, description='#3 The value of nd is less than 3', layout=Layout…

# PART B - Full Tutorial

## 1. Prepare sample data and model
### For the simplicity, we create simple 10-row data manually with two feature cols and one label col 

In [6]:
import pandas as pd
import numpy as np

lines_of_code = np.random.randint(500, size=10)
num_of_developers = np.random.randint(10, size=10)
labels = [True, True, True, True, True, False, False, True, True, False]
data = {'lines_of_code': lines_of_code, 'num_of_developers': num_of_developers, 'is_defect': labels}
df = pd.DataFrame(data=data)
df.head(3)

,lines_of_code,num_of_developers,is_defect
0,224,9,True
1,57,3,True
2,415,6,True


## Define features and label

In [146]:
feature_cols = ['lines_of_code', 'num_of_developers']
label_col = 'is_defect' 
X = df[feature_cols]
y = df[label_col]
print(X.head(3))
print()
print(y.head(3))

   lines_of_code  num_of_developers
0            224                  9
1             57                  3
2            415                  6

0    True
1    True
2    True
Name: is_defect, dtype: bool


## Split data into training and testing set

In [147]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

## Train a RandomForest model using sklearn

In [148]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)

rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

## Generate predictions

In [202]:
# generate prediction from the model, which will return a list of predicted labels
y_preds = rf_model.predict(X_test)
# store labels into a dictionary with preferred column name as key 
data = {'is_defect': y_preds}
# create a DataFrame which only has one column, label
y_preds = pd.DataFrame(data=data)
y_preds

,is_defect
0,True
1,False
2,True


## Select one row from testing set to be explained

In [205]:
# decide which row to be selected
selected_row = 0
# select the row in X_test which contains all of the feature values
X_explain = X_test.iloc[[selected_row]]
# select the corresponding label from the DataFrame that we just created above
y_explain = y_preds['is_defect'].iloc[[selected_row]]
print('one row of feature:', '\n\n', X_explain, '\n')
print('one row of label:', '\n\n', y_explain)

one row of feature: 

    lines_of_code  num_of_developers
4             93                  5 

one row of label: 

 0    True
Name: is_defect, dtype: bool


## Initialise a PyExplainer object

In [168]:
from pyexplainer import pyexplainer_pyexplainer

py_explainer = pyexplainer_pyexplainer.PyExplainer(X_train = X_train,
                                                   y_train = y_train,
                                                   indep = X_train.columns,
                                                   dep = 'is_defect',
                                                   blackbox_model = rf_model)

## Create rules by triggering explain function under PyExplainer object

In [ ]:
rules = py_explainer.explain(X_explain=X_explain,
                             y_explain=y_explain,
                             search_function='crossoverinterpolation')

#### Those created rules are stored in a dictionary, for more information about what is contained in each key, please refer to 'Appendix' part

In [173]:
rules.keys()

dict_keys(['synthetic_data', 'synthetic_predictions', 'X_explain', 'y_explain', 'indep', 'dep', 'top_k_positive_rules', 'top_k_negative_rules', 'local_rulefit_model'])

In [198]:
top_k_positive_rules = rules['top_k_positive_rules']

top_k_positive_rules = top_k_positive_rules.dropna()
print(top_k_positive_rules)
for i in range(len(top_k_positive_rules)):
    try:
        tmp_rule = (top_k_positive_rules['rule'].iloc[i])
        tmp_rule = tmp_rule.strip()
        tmp_rule = str.split(tmp_rule, '&')
    except:
        print(type(tmp_rule))

     index                                               rule  type  \
0      798  num_of_developers <= 3.0199999809265137 & line...  rule   
1      664  lines_of_code <= 479.885009765625 & lines_of_c...  rule   
2      846  lines_of_code <= 453.49000549316406 & lines_of...  rule   
3      932  lines_of_code <= 453.49000549316406 & lines_of...  rule   
4      929  lines_of_code > 467.55499267578125 & num_of_de...  rule   
..     ...                                                ...   ...   
144    455            num_of_developers <= -6.680000066757202  rule   
145    239  lines_of_code <= 451.77000427246094 & num_of_d...  rule   
146    698            num_of_developers <= -7.724999904632568  rule   
147    697  lines_of_code > 464.9949951171875 & lines_of_c...  rule   
148    111  num_of_developers <= 2.1450001001358032 & num_...  rule   

             coef   support    importance is_satisfy_instance  
0    9.176700e-01  0.279891  4.119839e-01                True  
1    7.063464e-01  

In [187]:
py_explainer.visualise(rules)

AttributeError: 'float' object has no attribute 'strip'

# Appendix

## The detail of variables used to to create PyExplainer

### Synthetic_data

Synthetic_data is data that are generated by PyExplainer using one of the following approaches.

1. Crossover and Interpolation
2. Random Perturbation.

After Synthetic_data is generated, it is stored as a pandas DataFrame object. 

In [17]:
print("Type of pyExp_rule_obj['synthetic_data'] - ", type(created_rule_obj['synthetic_data']), "\n")

print('Example')
display(created_rule_obj['synthetic_data'].head(2))

Type of pyExp_rule_obj['synthetic_data'] -  <class 'pandas.core.frame.DataFrame'> 

Example


,la,nd,ns,ent,nrev,rtime,self,ndev,age,app,rrexp,asawr,rsawr
0,194.0,2.0,1.0,0.80,12.0,9.40,0.0,44.0,1.97,2.0,1290.0,0.01,0.61
1,130.0,4.0,1.0,0.83,15.0,14.47,0.0,55.0,0.06,2.0,1206.0,0.00,0.54


### Synthetic_predictions

Synthetic_predictions is the prediction of Synthetic_data, which is obtained from the global model inside PyExplainer.

In [12]:
print("Type of pyExp_rule_obj['synthetic_predictions'] - ", type(created_rule_obj['synthetic_predictions']), "\n")
print("Example", "\n\n", created_rule_obj['synthetic_predictions'])

Type of pyExp_rule_obj['synthetic_predictions'] -  <class 'numpy.ndarray'> 

Example 

 [False  True False ...  True  True  True]


### X_explain

X_explain is an instance to be explained (which is a defective commit in this context)

In [15]:
print("Type of pyExp_rule_obj['X_explain'] - ", type(created_rule_obj['X_explain']), "\n")

print('Example')
display(created_rule_obj['X_explain'])

Type of pyExp_rule_obj['X_explain'] -  <class 'pandas.core.frame.DataFrame'> 

Example


,la,nd,ns,ent,nrev,rtime,self,ndev,age,app,rrexp,asawr,rsawr
commit_id,,,,,,,,,,,,,
a9a59ccbacafd6eb94f57861cfc28f5a24f474db,155,3,1,0.736602,12.0,32.516586,0,70.0,0.669703,3.0,2374.0,0.143199,0.390069


### y_explain

y_explain is a label of X_explain 

In [18]:
print("Type of pyExp_rule_obj['y_explain'] - ", type(created_rule_obj['y_explain']), "\n")
print("Example", "\n\n", created_rule_obj['y_explain'])

Type of pyExp_rule_obj['y_explain'] -  <class 'pandas.core.series.Series'> 

Example 

 commit_id
a9a59ccbacafd6eb94f57861cfc28f5a24f474db    True
Name: defect, dtype: bool


### indep
#### indep is feature names of X_explain

In [19]:
print("Type of pyExp_rule_obj['indep'] - ", type(created_rule_obj['indep']), "\n")
print("Example", "\n\n", created_rule_obj['indep'])

Type of pyExp_rule_obj['indep'] -  <class 'pandas.core.indexes.base.Index'> 

Example 

 Index(['la', 'nd', 'ns', 'ent', 'nrev', 'rtime', 'self', 'ndev', 'age', 'app',
       'rrexp', 'asawr', 'rsawr'],
      dtype='object')


### dep
#### dep is a label name

In [20]:
print("Type of pyExp_rule_obj['dep'] - ", type(created_rule_obj['dep']), "\n")
print("Example", "\n\n", created_rule_obj['dep'])

Type of pyExp_rule_obj['dep'] -  <class 'str'> 

Example 

 defect


### top_k_positive_rules

top_k_positive_rules is top-k rules that are genereated by PyExplainer to explain why a commit is predicted as defective.

Here we show top-3 rules that lead to defective commits=

In [26]:
print("Type of pyExp_rule_obj['top_k_positive_rules'] - ", type(created_rule_obj['top_k_positive_rules']), "\n")
print('Example')
display(created_rule_obj['top_k_positive_rules'].head(3))

Type of pyExp_rule_obj['top_k_positive_rules'] -  <class 'pandas.core.frame.DataFrame'> 

Example


,index,rule,type,coef,support,importance,is_satisfy_instance
0,572,la <= 104.84500122070312 & ndev > 71.319999694...,rule,0.220501,0.232376,0.093128,True
1,359,la > 103.68999862670898 & nrev > 8.11499977111...,rule,0.167275,0.237598,0.071194,True
2,98,app <= 3.9550000429153442 & ndev <= 103.875 & ...,rule,0.136704,0.631854,0.065933,True


### top_k_negative_rules

top_k_negative_rules is top-k negative rules that are genereated by PyExplainer to explain why a commit is predicted as clean.

The default number of generated rules is 3.


In [27]:
print("Type of pyExp_rule_obj['top_k_negative_rules'] - ", type(created_rule_obj['top_k_negative_rules']), "\n")
print('Example')
display(created_rule_obj['top_k_negative_rules'])

Type of pyExp_rule_obj['top_k_negative_rules'] -  <class 'pandas.core.frame.DataFrame'> 

Example


,rule,type,coef,support,importance,Class
820,ent <= 0.9350000023841858 & app > 2.9900000095...,rule,-0.216263,0.389034,0.105435,Clean
1609,app <= 4.009999990463257 & app > 2.99000000953...,rule,-0.206549,0.428198,0.102204,Clean
323,la <= 104.84500122070312 & ndev <= 71.31999969...,rule,-0.173105,0.391645,0.084496,Clean


# Bug Report Channel
#### Please report <a href="https://github.com/awsm-research/pyExplainer/issues">here</a>
#### 📧 or email your report to michaelfu1998@gmail.com